Seed Setting and Saver Function

In [ ]:
import numpy as np
import random
import torch

import os
# -----------------
# Global Seed Setting
# -----------------
SEED = 42

# Python built-in
random.seed(SEED)

# NumPy
np.random.seed(SEED)

# PyTorch
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

# Ensure deterministic runs
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

def save_df_to_downloads(df_name: str):
    if df_name in globals():
        df = globals()[df_name]
        path = f"/mnt/c/Users/baris/Downloads/{df_name}.pkl"
        df.to_pickle(path)
        print(f" Saved {df_name} to {path}")
    else:
        print(f" No DataFrame named '{df_name}' found in globals().")


Data Loader and Holiday Definitions

In [ ]:
# Load your ATM data
atm_path = "/mnt/c/Users/baris/Downloads/2024-12-09_ATM_Branch_Data.xlsx"
atm = pd.read_excel(atm_path, sheet_name="ATM")

# Official Holidays (special_days)
special_days = [
    # 2006
    '2006-01-01', '2006-01-10', '2006-01-11', '2006-01-12', '2006-01-13',
    '2006-04-23', '2006-05-19', '2006-08-30',
    '2006-10-23', '2006-10-24', '2006-10-25',
    '2006-10-29', '2006-12-31',

    # 2007
    '2007-01-01', '2007-01-02', '2007-01-03',
    '2007-04-23', '2007-05-19', '2007-08-30',
    '2007-10-12', '2007-10-13', '2007-10-14',
    '2007-10-29',
    '2007-12-20', '2007-12-21', '2007-12-22', '2007-12-23',
    '2007-12-31',

    # 2008
    '2008-01-01', '2008-04-23', '2008-05-19', '2008-08-30',
    '2008-09-30', '2008-10-01', '2008-10-02',
    '2008-10-29',
    '2008-12-08', '2008-12-09', '2008-12-10', '2008-12-11',
    '2008-12-31'
]

# Half Working Days (half_days)
half_days = [
    '2006-01-09', '2006-10-22', '2006-12-30',
    '2007-10-11', '2007-12-19',
    '2008-09-29', '2008-12-07'
]

feature_eng_data = feature_engineering_pipeline(atm, special_days, half_days)

print(feature_eng_data)
print(feature_eng_data.columns)

Calendar and Holiday Features Creation Functions

In [ ]:
import pandas as pd
import numpy as np

# -----------------
# Calendar Features
# -----------------
def add_calendar_features(df):
    df["day_of_month"] = df["DATE"].dt.day
    df["month"] = df["DATE"].dt.month
    df["day_of_week"] = df["DATE"].dt.dayofweek 

    # --- Day of month dummies ---
    dom_dummies = pd.get_dummies(df["day_of_month"], prefix="dom")
    dom_dummies = dom_dummies.drop(columns=["dom_1"], errors="ignore")

    # --- Month dummies ---
    month_dummies = pd.get_dummies(df["month"], prefix="month")
    month_dummies = month_dummies.drop(columns=["month_12"], errors="ignore")

    # --- Day of week dummies ---
    dow_dummies = pd.get_dummies(df["day_of_week"], prefix="dow")
    dow_dummies = dow_dummies.drop(columns=["dow_0"], errors="ignore")  

    # Drop raw columns
    df = df.drop(columns=["day_of_month", "month", "day_of_week"])
    df = pd.concat([df, dom_dummies, month_dummies, dow_dummies], axis=1)

    return df

# -----------------
# Holiday Features
# -----------------
def add_holiday_features(df, official_holidays, half_days):
    df["DATE"] = pd.to_datetime(df["DATE"])
    official_holidays = pd.to_datetime(official_holidays)
    half_days = pd.to_datetime(half_days)

    df["day_official_holiday"] = df["DATE"].isin(official_holidays).astype(int)
    df["day_half_day"] = df["DATE"].isin(half_days).astype(int)

    # Everything else = normal day
    df["day_normal_day"] = (
        (df["day_official_holiday"] == 0) &
        (df["day_half_day"] == 0)
    ).astype(int)
    return df

# -----------------
# Final Wrapper
# -----------------
def feature_engineering_pipeline(atm, official_holidays, half_days):
    df = atm.copy()
    df = add_calendar_features(df)  
    df = add_holiday_features(df, official_holidays, half_days)
    return df.reset_index(drop=True)

Helper Functions (Clustering, Data Prep, Forecast Metrics, Normalization, Reverse Transformation)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

# -----------------
# 1. Data Preparation
# -----------------
def prepare_data(df, test_size=0.2):
    """
    Prepare features (X) and target (y), then split train/test by time.
    Target is kept raw (WITHDRWLS).
    Normalization must be applied later with normalize_target().
    """
    feature_cols = [
        col for col in df.columns
        if col not in [
            "CASHP_ID", "DATE", "WITHDRWLS",
            "day_of_week", "day_of_month",
            "is_active", "in_active", "cluster"  # exclude cluster
        ]
    ]
    X = df[feature_cols]
    y = df["WITHDRWLS"]

    split_idx = int(len(df) * (1 - test_size))
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

    return X_train, X_test, y_train, y_test

# -----------------
# 2. ATM Clustering (range of clusters)
# -----------------
def cluster_atms_range(df, cluster_range=(1, 4), zero_run_threshold=14):
    """
    Cluster ATMs for a range of cluster sizes.
    Uses withdrawals with zeros allowed, but discards sequences of ≥zero_run_threshold consecutive zeros.
    Returns dict {n_clusters: (clustered_df, kmeans_model)}.
    """
    results = []

    # Step 1: Drop sequences of consecutive zeros ≥ threshold
    def drop_long_zero_runs(group):
        mask = np.ones(len(group), dtype=bool)  # True = keep, False = drop
        zero_count = 0

        for i, val in enumerate(group["WITHDRWLS"].values):
            if val == 0:
                zero_count += 1
            else:
                if zero_count >= zero_run_threshold:
                    mask[i - zero_count:i] = False  # discard long zero run
                zero_count = 0

        # Handle case where series ends with zeros
        if zero_count >= zero_run_threshold:
            mask[len(group) - zero_count:] = False

        return group[mask]

    filtered_df = df.groupby("CASHP_ID", group_keys=False).apply(drop_long_zero_runs)

    # Step 2: ATM profiles based on filtered withdrawals
    atm_profiles = filtered_df.groupby("CASHP_ID")["WITHDRWLS"].mean().to_frame("avg_withdrawals")

    # Step 3: Clustering for each cluster size
    results = {}
    for n in range(cluster_range[0], cluster_range[1] + 1):
        kmeans = KMeans(n_clusters=n, random_state=42, n_init=10)
        atm_profiles[f"cluster_{n}"] = kmeans.fit_predict(atm_profiles[["avg_withdrawals"]])

        # Merge cluster labels back to the *filtered* dataset
        df_clustered = filtered_df.merge(
            atm_profiles[[f"cluster_{n}"]], on="CASHP_ID", how="left"
        )
        df_clustered = df_clustered.rename(columns={f"cluster_{n}": "cluster"})

        results[n] = (df_clustered, kmeans)

    return results
# -----------------
# 3. Forecast Metrics
# -----------------
def calculate_forecast_metrics(y_true, y_pred, model_name="Model"):
    """
    Calculate MAE, RMSE, MAPE, and SMAPE for forecasting.
    Metrics are calculated in raw units.
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)

    mae = np.mean(np.abs(y_true - y_pred))
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-9))) * 100
    smape = np.mean(2 * np.abs(y_pred - y_true) /
                    (np.abs(y_true) + np.abs(y_pred) + 1e-9)) * 100

    results = {
        "Model": model_name,
        "MAE": mae,
        "RMSE": rmse,
        "MAPE": mape,
        "SMAPE": smape
    }

    return results

# -----------------
# 4. Normalization (target only, applied after split)
# -----------------
def normalize_target(y_train, y_test, method="log"):
    """
    Normalize target values after train/test split.
    Prevents data leakage by fitting only on training set.
    
    Returns:
        y_train_norm, y_test_norm, scaler
    """
    if method == "log":
        y_train_norm = np.log1p(y_train)
        y_test_norm  = np.log1p(y_test)
        scaler = None

    elif method == "minmax":
        scaler = MinMaxScaler()
        y_train_norm = scaler.fit_transform(y_train.to_frame()).ravel()
        y_test_norm  = scaler.transform(y_test.to_frame()).ravel()

    else:
        raise ValueError("method must be 'log' or 'minmax'")

    return y_train_norm, y_test_norm, scaler

# -----------------
# 5. Reverse transform predictions
# -----------------
def inverse_transform_predictions(y_pred_norm, method="log", scaler=None):
    """
    Reverse transform model predictions back to raw scale.
    Uses the same method as in normalize_target.
    """
    if method == "log":
        return np.expm1(y_pred_norm)
    elif method == "minmax" and scaler is not None:
        return scaler.inverse_transform(y_pred_norm.reshape(-1,1)).ravel()
    else:
        return y_pred_norm




LSTM with Feature Set 0 (No Feature Set)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import optuna
import pandas as pd

# -----------------
# Dataset (target-only)
# -----------------
class SeqOnlyDataset(Dataset):
    def __init__(self, series, seq_len=30):
        self.samples = []
        series = np.array(series, dtype=np.float32)

        for i in range(len(series) - seq_len):
            X_seq = series[i:i+seq_len].reshape(-1, 1) 
            y_seq = series[i+seq_len]

            # Skip if *any* value in the sequence OR the target is 0
            if np.any(X_seq == 0) or y_seq == 0:
                continue

            self.samples.append((X_seq, y_seq))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        X_seq, y_seq = self.samples[idx]
        return (
            torch.tensor(X_seq, dtype=torch.float32),
            torch.tensor(y_seq, dtype=torch.float32),
        )

# -----------------
# LSTM Model (Univariate)
# -----------------
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_dim, hidden_dim, num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0
        )
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  
        return out.view(-1)         
        
# -----------------
# Runner: LSTM Univariate + Clustering + Optuna
# -----------------
def run_lstm_univariate_with_clustering(df, cluster_range=(1,4), seq_lens=[7,14,30], n_trials=30, norm_method="minmax"):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    all_metrics = []

    clustered_results = cluster_atms_range(df, cluster_range=cluster_range)

    for n_clusters, (df_clustered, _) in clustered_results.items():
        print(f"\n=== LSTM (Univariate) with {n_clusters} Clusters ===")

        for seq_len in seq_lens:
            print(f"\n--- Seq_len={seq_len} ---")

            for cluster_id in sorted(df_clustered["cluster"].dropna().unique()):
                cluster_df = df_clustered[df_clustered["cluster"] == cluster_id].copy()

                # --- Use prepare_data for time split ---
                _, _, y_train, y_val = prepare_data(cluster_df, test_size=0.2)

                # --- Normalize target ---
                y_train_norm, y_val_norm, scaler = normalize_target(y_train, y_val, method=norm_method)

                trial_metrics = []

                def objective(trial):
                    hidden_dim = trial.suggest_int("hidden_dim", 16, 128, step=16)
                    num_layers = trial.suggest_int("num_layers", 1, 3)
                    dropout    = trial.suggest_float("dropout", 0.0, 0.5)
                    lr         = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
                    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])

                    train_dataset = SeqOnlyDataset(y_train_norm, seq_len)
                    val_dataset   = SeqOnlyDataset(y_val_norm, seq_len)

                    if len(train_dataset) == 0 or len(val_dataset) == 0:
                        print(f"Skipping: cluster={cluster_id}, seq_len={seq_len}, trial={trial.number} (no valid samples)")
                        return float("inf")

                    train_loader  = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader    = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

                    model = LSTMModel(
                        input_dim=1,
                        hidden_dim=hidden_dim,
                        num_layers=num_layers,
                        dropout=dropout
                    ).to(device)

                    criterion = nn.MSELoss()
                    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

                    # Training
                    for _ in range(20):
                        model.train()
                        for X_batch, y_batch in train_loader:
                            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                            optimizer.zero_grad()
                            preds = model(X_batch)
                            loss = criterion(preds, y_batch)
                            loss.backward()
                            optimizer.step()

                    # Validation
                    model.eval()
                    preds_norm, trues_norm = [], []
                    with torch.no_grad():
                        for X_batch, y_batch in val_loader:
                            X_batch = X_batch.to(device)
                            out = model(X_batch).cpu().numpy()
                            preds_norm.extend(out)
                            trues_norm.extend(y_batch.numpy())

                    preds = inverse_transform_predictions(
                        np.array(preds_norm), method=norm_method, scaler=scaler
                    )
                    trues = inverse_transform_predictions(
                        np.array(trues_norm), method=norm_method, scaler=scaler
                    )

                    metrics = calculate_forecast_metrics(
                        trues, preds, model_name=f"LSTM trial {trial.number}"
                    )
                    metrics["trial"] = trial.number
                    trial_metrics.append(metrics)

                    return metrics["SMAPE"]

                # Run Optuna
                study = optuna.create_study(direction="minimize")
                study.optimize(objective, n_trials=n_trials)

                if len(trial_metrics) == 0:
                    print(f"⚠️ No valid trials for cluster={cluster_id}, seq_len={seq_len}, n_clusters={n_clusters}")
                    continue

                # Best trial
                best_trial = study.best_trial.number
                best_metrics = [m for m in trial_metrics if m["trial"] == best_trial][0]
                best_metrics.update({
                    "Seq_len": seq_len,
                    "Best_Params": study.best_trial.params,
                    "n_clusters": n_clusters,
                    "cluster_id": cluster_id
                })

                all_metrics.append(best_metrics)

    return pd.DataFrame(all_metrics)

# -----------------
# Run
# -----------------
no_feature_lstm = run_lstm_univariate_with_clustering(
    feature_eng_data,
    cluster_range=(1,4),
    seq_lens=[7, 14, 30, 90],
    n_trials=20,
    norm_method="log"   # or "minmax"
)

print(no_feature_lstm)
save_df_to_downloads("no_feature_lstm")

N-Beats with Feature Set 0 (No Feature Set)

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import optuna

# -----------------
# Dataset (target-only, drops zero sequences)
# -----------------
class SeqDataset(Dataset):
    def __init__(self, y, seq_len=30):
        self.samples = []
        y = np.array(y, dtype=np.float32)

        for i in range(len(y) - seq_len):
            X_seq = y[i:i + seq_len]
            y_seq = y[i + seq_len]

            # Drop if target == 0
            if y_seq == 0:
                continue

            self.samples.append((X_seq, y_seq))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        X_seq, y_seq = self.samples[idx]
        return (
            torch.tensor(X_seq, dtype=torch.float32).view(-1),
            torch.tensor(y_seq, dtype=torch.float32),
        )

# -----------------
# N-BEATS Model (Univariate)
# -----------------
class NBeatsBlock(nn.Module):
    def __init__(self, hidden_dim, theta_dim, backcast_length, forecast_length):
        super().__init__()
        self.fc1 = nn.Linear(backcast_length, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, theta_dim)
        self.backcast_length = backcast_length
        self.forecast_length = forecast_length

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        theta = self.fc4(x)
        backcast = theta[:, :self.backcast_length]
        forecast = theta[:, -self.forecast_length:]
        return backcast, forecast


class NBeats(nn.Module):
    def __init__(self, hidden_dim, backcast_length, forecast_length, n_blocks=3):
        super().__init__()
        theta_dim = backcast_length + forecast_length
        self.blocks = nn.ModuleList([
            NBeatsBlock(hidden_dim, theta_dim, backcast_length, forecast_length)
            for _ in range(n_blocks)
        ])
        self.forecast_length = forecast_length

    def forward(self, x):
        backcast = x.view(x.size(0), -1)
        forecast = torch.zeros(x.size(0), self.forecast_length, device=x.device)
        for block in self.blocks:
            b, f = block(backcast)
            backcast = backcast - b
            forecast = forecast + f
        return forecast.squeeze(-1)

# -----------------
# Runner with Optuna
# -----------------
def run_nbeats_univariate_with_clustering(df, cluster_range=(1, 4),
                                          seq_lens=[7, 14, 30],
                                          norm_method="minmax",
                                          n_trials=30):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    all_metrics = []

    clustered_results = cluster_atms_range(df, cluster_range=cluster_range)

    for n_clusters, (df_clustered, _) in clustered_results.items():
        print(f"\n=== N-BEATS (Univariate) with {n_clusters} Clusters ===")

        for seq_len in seq_lens:
            print(f"\n--- Seq_len={seq_len} ---")

            for cluster_id in sorted(df_clustered["cluster"].dropna().unique()):
                cluster_df = df_clustered[df_clustered["cluster"] == cluster_id].copy()

                # --- Use prepare_data for consistent split ---
                _, _, y_train, y_val = prepare_data(cluster_df, test_size=0.2)

                # --- Normalize target ---
                y_train_norm, y_val_norm, scaler = normalize_target(y_train, y_val, method=norm_method)

                train_dataset = SeqDataset(y_train_norm, seq_len)
                val_dataset   = SeqDataset(y_val_norm, seq_len)

                if len(train_dataset) == 0 or len(val_dataset) == 0:
                    print(f" Cluster {cluster_id}, seq_len {seq_len} has no valid (nonzero) sequences. Skipping.")
                    continue

                # -----------------
                # Optuna objective
                # -----------------
                def objective(trial):
                    hidden_dim = trial.suggest_int("hidden_dim", 32, 256, step=32)
                    n_blocks   = trial.suggest_int("n_blocks", 2, 6)
                    lr         = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
                    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])

                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

                    model = NBeats(hidden_dim=hidden_dim,
                                   backcast_length=seq_len,
                                   forecast_length=1,
                                   n_blocks=n_blocks).to(device)
                    criterion = nn.MSELoss()
                    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

                    # Training
                    for _ in range(20):
                        model.train()
                        for X_batch, y_batch in train_loader:
                            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                            optimizer.zero_grad()
                            preds = model(X_batch)
                            loss = criterion(preds, y_batch)
                            loss.backward()
                            optimizer.step()

                    # Validation
                    model.eval()
                    preds_norm, trues_norm = [], []
                    with torch.no_grad():
                        for X_batch, y_batch in val_loader:
                            X_batch = X_batch.to(device)
                            out = model(X_batch).cpu().numpy()
                            preds_norm.extend(out)
                            trues_norm.extend(y_batch.numpy())

                    preds = inverse_transform_predictions(np.array(preds_norm), method=norm_method, scaler=scaler)
                    trues = inverse_transform_predictions(np.array(trues_norm), method=norm_method, scaler=scaler)

                    smape = 100 * np.mean(2 * np.abs(preds - trues) / (np.abs(preds) + np.abs(trues) + 1e-9))
                    return smape

                # Run Optuna
                study = optuna.create_study(direction="minimize")
                study.optimize(objective, n_trials=n_trials, show_progress_bar=False)

                best_params = study.best_trial.params

                # -----------------
                # Retrain with best params
                # -----------------
                train_loader = DataLoader(train_dataset, batch_size=best_params["batch_size"], shuffle=True)
                val_loader   = DataLoader(val_dataset, batch_size=best_params["batch_size"], shuffle=False)

                model = NBeats(hidden_dim=best_params["hidden_dim"],
                               backcast_length=seq_len,
                               forecast_length=1,
                               n_blocks=best_params["n_blocks"]).to(device)
                optimizer = torch.optim.Adam(model.parameters(), lr=best_params["lr"])
                criterion = nn.MSELoss()

                for _ in range(20):
                    model.train()
                    for X_batch, y_batch in train_loader:
                        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                        optimizer.zero_grad()
                        preds = model(X_batch)
                        loss = criterion(preds, y_batch)
                        loss.backward()
                        optimizer.step()

                # Final validation
                model.eval()
                preds_norm, trues_norm = [], []
                with torch.no_grad():
                    for X_batch, y_batch in val_loader:
                        X_batch = X_batch.to(device)
                        out = model(X_batch).cpu().numpy()
                        preds_norm.extend(out)
                        trues_norm.extend(y_batch.numpy())

                preds = inverse_transform_predictions(np.array(preds_norm), method=norm_method, scaler=scaler)
                trues = inverse_transform_predictions(np.array(trues_norm), method=norm_method, scaler=scaler)

                metrics = calculate_forecast_metrics(
                    trues, preds,
                    model_name=f"N-BEATS-Uni (k={n_clusters}, c={cluster_id}, seq={seq_len})"
                )
                metrics.update({
                    "Seq_len": seq_len,
                    "n_clusters": n_clusters,
                    "cluster_id": cluster_id,
                    "Best_Params": best_params
                })

                all_metrics.append(metrics)

    return pd.DataFrame(all_metrics)

# -----------------
# Example Run
# -----------------
no_feature_nbeats = run_nbeats_univariate_with_clustering(
    feature_eng_data,
    cluster_range=(1, 4),
    seq_lens=[7, 14,30,90],
    norm_method="log",
    n_trials=20
)

print(no_feature_nbeats)
save_df_to_downloads("no_feature_nbeats")

Linear Regression with Feature Set 1 (Dummy Variables)

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd

def run_linear_regression_per_cluster(df, cluster_range=(1,4), test_size=0.2, norm_method="log"):
    # Drop zero withdrawals globally
    df = df[df["WITHDRWLS"] > 0].copy()

    all_metrics = {}
    clustered_results = cluster_atms_range(df, cluster_range=cluster_range)

    for n_clusters, (df_clustered, _) in clustered_results.items():
        print(f"\n=== Linear Regression with {n_clusters} Clusters ===")

        for cluster_id in sorted(df_clustered["cluster"].dropna().unique()):
            cluster_df = df_clustered[df_clustered["cluster"] == cluster_id].copy()
            if cluster_df.empty:
                print(f"Cluster {cluster_id} has no valid rows. Skipping.")
                continue

            # --- Use unified prepare_data ---
            X_train, X_test, y_train, y_test = prepare_data(cluster_df, test_size=test_size)

            # --- Normalize AFTER split ---
            y_train_norm, y_test_norm, scaler = normalize_target(y_train, y_test, method=norm_method)

            # --- Train ---
            model = LinearRegression()
            model.fit(X_train, y_train_norm)

            # --- Predict & inverse transform ---
            y_pred_norm = model.predict(X_test)
            y_pred_raw = inverse_transform_predictions(y_pred_norm, method=norm_method, scaler=scaler)

            # --- Evaluate ---
            metrics = calculate_forecast_metrics(
                y_test, y_pred_raw,
                model_name=f"LR (k={n_clusters}, cluster={cluster_id})"
            )
            metrics["n_clusters"] = n_clusters
            metrics["cluster_id"] = cluster_id

            all_metrics.setdefault(n_clusters, []).append(metrics)

    # Flatten results
    results = [m for cluster_metrics in all_metrics.values() for m in cluster_metrics]
    return pd.DataFrame(results)


# --- Run example ---
dummy_var_lin_reg = run_linear_regression_per_cluster(
    feature_eng_data,
    cluster_range=(1,4),
    test_size=0.2,
    norm_method="log"
)

print(dummy_var_lin_reg)
save_df_to_downloads("dummy_var_lin_reg")


XGBoost with Feature Set 1 (Dummy Variables)

In [ ]:
import xgboost as xgb
import optuna
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error

def run_xgboost_per_cluster(df, cluster_range=(1,4), test_size=0.2, method="log", n_trials=50):

    # Drop zero withdrawals globally
    df = df[df["WITHDRWLS"] > 0].copy()

    all_metrics = []
    optuna.logging.set_verbosity(optuna.logging.WARNING)

    clustered_results = cluster_atms_range(df, cluster_range=cluster_range)

    for n_clusters, (df_clustered, _) in clustered_results.items():
        print(f"\n=== XGBoost with {n_clusters} Clusters ===")

        for cluster_id in sorted(df_clustered["cluster"].dropna().unique()):
            cluster_df = df_clustered[df_clustered["cluster"] == cluster_id].copy()
            if cluster_df.empty:
                print(f"Cluster {cluster_id} has no valid rows. Skipping.")
                continue

            # --- Use unified prepare_data ---
            X_train, X_val, y_train, y_val = prepare_data(cluster_df, test_size=test_size)

            # --- Normalize AFTER split ---
            y_train_norm, y_val_norm, scaler = normalize_target(y_train, y_val, method=method)

            # --- Objective for Optuna ---
            def objective(trial):
                params = {
                    "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
                    "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.1, log=True),
                    "max_depth": trial.suggest_int("max_depth", 3, 10),
                    "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
                    "gamma": trial.suggest_float("gamma", 0, 5),
                    "subsample": trial.suggest_float("subsample", 0.5, 1.0),
                    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
                    "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
                    "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
                    "random_state": 42,
                    "tree_method": "hist"
                }

                model = xgb.XGBRegressor(**params)
                model.fit(X_train, y_train_norm,
                          eval_set=[(X_val, y_val_norm)],
                          verbose=False)

                # Predict & inverse transform
                y_pred_norm = model.predict(X_val)
                y_pred_raw = inverse_transform_predictions(y_pred_norm, method=method, scaler=scaler)

                # Evaluate
                metrics = calculate_forecast_metrics(
                    y_val, y_pred_raw,
                    model_name=f"XGB (k={n_clusters}, cluster={cluster_id})"
                )
                trial.set_user_attr("mae", metrics["MAE"])
                return metrics["SMAPE"]

            # Run Optuna
            study = optuna.create_study(direction="minimize")
            study.optimize(objective, n_trials=n_trials)

            # Collect best results
            best_smape = study.best_value
            best_mae   = study.best_trial.user_attrs["mae"]
            best_params = study.best_trial.params

            print(f"Cluster {cluster_id} | Best SMAPE={best_smape:.2f}% | MAE={best_mae:.2f}")

            all_metrics.append({
                "Model": f"XGB (k={n_clusters}, cluster={cluster_id})",
                "Best_SMAPE": best_smape,
                "Best_MAE": best_mae,
                "Best_Params": best_params,
                "n_clusters": n_clusters,
                "cluster_id": cluster_id
            })

    return pd.DataFrame(all_metrics)


# Run
dummy_var_xgb = run_xgboost_per_cluster(
    feature_eng_data,
    cluster_range=(1,4),
    test_size=0.2,
    method="log",
    n_trials=100
)

print(dummy_var_xgb)
save_df_to_downloads("dummy_var_xgb")


LSTM with Feature Set 1 (Dummy Variables)

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import optuna

# -----------------
# Dataset Class (drops sequences with zeros)
# -----------------
class SeqDataset(Dataset):
    def __init__(self, X, y, seq_len=30):
        self.samples = []
        X = np.array(X, dtype=np.float32)
        y = np.array(y, dtype=np.float32)

        for i in range(len(X) - seq_len):
            X_seq = X[i:i+seq_len]
            y_seq = y[i+seq_len]

            if y_seq == 0:
                continue
            self.samples.append((X_seq, y_seq))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        X_seq, y_seq = self.samples[idx]
        return torch.tensor(X_seq, dtype=torch.float32), torch.tensor(y_seq, dtype=torch.float32)

# -----------------
# LSTM Model
# -----------------
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=2, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers,
                            batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :]) 
        return out.view(-1)  

# -----------------
# Runner (with Optuna + dropping zeros)
# -----------------
def run_lstm_with_clustering(df, cluster_range=(1,4), seq_lens=[7,14,30], norm_method="minmax", n_trials=30):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    all_metrics = []

    clustered_results = cluster_atms_range(df, cluster_range=cluster_range)

    for n_clusters, (df_clustered, _) in clustered_results.items():
        print(f"\n=== LSTM with {n_clusters} Clusters ===")

        for seq_len in seq_lens:
            print(f"\n--- Seq_len={seq_len} ---")

            for cluster_id in sorted(df_clustered["cluster"].dropna().unique()):
                cluster_df = df_clustered[df_clustered["cluster"] == cluster_id].copy()

                # --- Train/val split
                X_train, X_val, y_train, y_val = prepare_data(cluster_df, test_size=0.2)
                X_train, X_val = X_train.astype(np.float32), X_val.astype(np.float32)

                # --- Normalize target
                y_train_norm, y_val_norm, scaler = normalize_target(y_train, y_val, method=norm_method)

                # --- Build datasets (zeros dropped automatically inside SeqDataset)
                train_dataset = SeqDataset(X_train.values, y_train_norm, seq_len)
                val_dataset   = SeqDataset(X_val.values, y_val_norm, seq_len)

                if len(train_dataset) == 0 or len(val_dataset) == 0:
                    print(f"Cluster {cluster_id}, seq_len {seq_len} has no valid (nonzero) sequences. Skipping.")
                    continue

                # -----------------
                # Optuna Objective
                # -----------------
                def objective(trial):
                    hidden_dim = trial.suggest_int("hidden_dim", 32, 128, step=16)
                    num_layers = trial.suggest_int("num_layers", 1, 3)
                    dropout    = trial.suggest_float("dropout", 0.0, 0.5)
                    lr         = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
                    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])

                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

                    model = LSTMModel(input_dim=X_train.shape[1], hidden_dim=hidden_dim,
                                      num_layers=num_layers, dropout=dropout).to(device)
                    criterion = nn.MSELoss()
                    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

                    # Training
                    for _ in range(20):
                        model.train()
                        for X_batch, y_batch in train_loader:
                            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                            optimizer.zero_grad()
                            preds = model(X_batch)
                            loss = criterion(preds, y_batch)
                            loss.backward()
                            optimizer.step()

                    # Validation
                    model.eval()
                    preds_norm, trues_norm = [], []
                    with torch.no_grad():
                        for X_batch, y_batch in val_loader:
                            X_batch = X_batch.to(device)
                            out = model(X_batch).cpu().numpy()
                            preds_norm.extend(out)
                            trues_norm.extend(y_batch.numpy())

                    preds = inverse_transform_predictions(np.array(preds_norm), method=norm_method, scaler=scaler)
                    trues = inverse_transform_predictions(np.array(trues_norm), method=norm_method, scaler=scaler)

                    smape = 100 * np.mean(2 * np.abs(preds - trues) / (np.abs(preds) + np.abs(trues) + 1e-9))
                    return smape

                # Run Optuna silently
                study = optuna.create_study(direction="minimize")
                study.optimize(objective, n_trials=n_trials, show_progress_bar=False)

                # Best params
                best_params = study.best_trial.params

                # Retrain with best params
                batch_size = best_params["batch_size"]
                train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

                model = LSTMModel(input_dim=X_train.shape[1],
                                  hidden_dim=best_params["hidden_dim"],
                                  num_layers=best_params["num_layers"],
                                  dropout=best_params["dropout"]).to(device)
                optimizer = torch.optim.Adam(model.parameters(), lr=best_params["lr"])
                criterion = nn.MSELoss()

                for _ in range(20):
                    model.train()
                    for X_batch, y_batch in train_loader:
                        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                        optimizer.zero_grad()
                        preds = model(X_batch)
                        loss = criterion(preds, y_batch)
                        loss.backward()
                        optimizer.step()

                # Final validation evaluation
                model.eval()
                preds_norm, trues_norm = [], []
                with torch.no_grad():
                    for X_batch, y_batch in val_loader:
                        X_batch = X_batch.to(device)
                        out = model(X_batch).cpu().numpy()
                        preds_norm.extend(out)
                        trues_norm.extend(y_batch.numpy())

                preds = inverse_transform_predictions(np.array(preds_norm), method=norm_method, scaler=scaler)
                trues = inverse_transform_predictions(np.array(trues_norm), method=norm_method, scaler=scaler)

                metrics = calculate_forecast_metrics(
                    trues, preds,
                    model_name=f"LSTM (k={n_clusters}, c={cluster_id}, seq={seq_len})"
                )
                metrics.update({
                    "Seq_len": seq_len,
                    "n_clusters": n_clusters,
                    "cluster_id": cluster_id,
                    "Best_Params": best_params
                })

                all_metrics.append(metrics)

    return pd.DataFrame(all_metrics)

# --- Run LSTM with dummy vars, dropping zero-contaminated sequences ---
dummy_var_lstm = run_lstm_with_clustering(
    feature_eng_data,          
    cluster_range=(1,4),       
    seq_lens=[7, 14,30,90],  
    norm_method="log",       
    n_trials=20                 
)

print(dummy_var_lstm)

# Save results if needed
save_df_to_downloads("dummy_var_lstm")

N-Beats with Feature Set 1 (Dummy Variables)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np
import optuna

# -----------------
# Dataset (drops sequences with zeros)
# -----------------
class SeqDataset(Dataset):
    def __init__(self, X, y, seq_len=30):
        self.samples = []
        X = np.array(X, dtype=np.float32)
        y = np.array(y, dtype=np.float32)

        for i in range(len(X) - seq_len):
            X_seq = X[i:i+seq_len]
            y_seq = y[i+seq_len]

            if y_seq == 0:
                continue

            self.samples.append((X_seq, y_seq))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        X_seq, y_seq = self.samples[idx]
        return torch.tensor(X_seq, dtype=torch.float32), torch.tensor(y_seq, dtype=torch.float32)

# -----------------
# N-BEATS Model
# -----------------
class NBeatsBlock(nn.Module):
    def __init__(self, input_dim, hidden_dim, theta_dim, backcast_length, forecast_length):
        super().__init__()
        self.fc1 = nn.Linear(backcast_length * input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, theta_dim)
        self.backcast_length = backcast_length
        self.forecast_length = forecast_length
        self.input_dim = input_dim

    def forward(self, x):
        x = x.reshape(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        theta = self.fc4(x)

        backcast = theta[:, :self.backcast_length * self.input_dim].reshape(
            x.size(0), self.backcast_length, self.input_dim
        )
        forecast = theta[:, -self.forecast_length:]
        return backcast, forecast

class NBeats(nn.Module):
    def __init__(self, input_dim, hidden_dim, backcast_length, forecast_length, n_blocks=3):
        super().__init__()
        theta_dim = backcast_length * input_dim + forecast_length
        self.blocks = nn.ModuleList([
            NBeatsBlock(input_dim, hidden_dim, theta_dim, backcast_length, forecast_length)
            for _ in range(n_blocks)
        ])

    def forward(self, x):
        backcast = x
        forecast = 0
        for block in self.blocks:
            b, f = block(backcast)
            backcast = backcast - b
            forecast = forecast + f
        return forecast.view(-1)

# -----------------
# Runner (with Optuna)
# -----------------
def run_nbeats_with_clustering(df, cluster_range=(1,4), seq_lens=[7,14,30], norm_method="minmax", n_trials=30):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    all_metrics = []

    clustered_results = cluster_atms_range(df, cluster_range=cluster_range)

    for n_clusters, (df_clustered, _) in clustered_results.items():
        print(f"\n=== N-BEATS with {n_clusters} Clusters ===")

        for seq_len in seq_lens:
            print(f"\n--- Seq_len={seq_len} ---")

            for cluster_id in sorted(df_clustered["cluster"].dropna().unique()):
                cluster_df = df_clustered[df_clustered["cluster"] == cluster_id].copy()

                # --- Train/val split
                X_train, X_val, y_train, y_val = prepare_data(cluster_df, test_size=0.2)
                X_train, X_val = X_train.astype(np.float32), X_val.astype(np.float32)

                # --- Normalize target
                y_train_norm, y_val_norm, scaler = normalize_target(y_train, y_val, method=norm_method)

                # --- Build datasets
                train_dataset = SeqDataset(X_train.values, y_train_norm, seq_len)
                val_dataset   = SeqDataset(X_val.values, y_val_norm, seq_len)

                if len(train_dataset) == 0 or len(val_dataset) == 0:
                    print(f"Cluster {cluster_id}, seq_len {seq_len} has no valid sequences. Skipping.")
                    continue

                # -----------------
                # Optuna Objective
                # -----------------
                def objective(trial):
                    hidden_dim = trial.suggest_int("hidden_dim", 32, 256, step=32)
                    n_blocks   = trial.suggest_int("n_blocks", 2, 6)
                    lr         = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
                    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])

                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

                    model = NBeats(input_dim=X_train.shape[1], hidden_dim=hidden_dim,
                                   backcast_length=seq_len, forecast_length=1,
                                   n_blocks=n_blocks).to(device)

                    criterion = nn.MSELoss()
                    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

                    # Training
                    for _ in range(20):
                        model.train()
                        for X_batch, y_batch in train_loader:
                            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                            optimizer.zero_grad()
                            preds = model(X_batch)
                            loss = criterion(preds, y_batch)
                            loss.backward()
                            optimizer.step()

                    # Validation
                    model.eval()
                    preds_norm, trues_norm = [], []
                    with torch.no_grad():
                        for X_batch, y_batch in val_loader:
                            X_batch = X_batch.to(device)
                            out = model(X_batch).cpu().numpy()
                            preds_norm.extend(out)
                            trues_norm.extend(y_batch.numpy())

                    preds = inverse_transform_predictions(np.array(preds_norm), method=norm_method, scaler=scaler)
                    trues = inverse_transform_predictions(np.array(trues_norm), method=norm_method, scaler=scaler)

                    smape = 100 * np.mean(2 * np.abs(preds - trues) / (np.abs(preds) + np.abs(trues) + 1e-9))
                    return smape

                # Run Optuna silently
                study = optuna.create_study(direction="minimize")
                study.optimize(objective, n_trials=n_trials, show_progress_bar=False)

                # Best trial
                best_params = study.best_trial.params

                # --- Retrain with best params
                train_loader = DataLoader(train_dataset, batch_size=best_params["batch_size"], shuffle=True)
                val_loader   = DataLoader(val_dataset, batch_size=best_params["batch_size"], shuffle=False)

                model = NBeats(input_dim=X_train.shape[1], hidden_dim=best_params["hidden_dim"],
                               backcast_length=seq_len, forecast_length=1,
                               n_blocks=best_params["n_blocks"]).to(device)
                optimizer = torch.optim.Adam(model.parameters(), lr=best_params["lr"])
                criterion = nn.MSELoss()

                for _ in range(20):
                    model.train()
                    for X_batch, y_batch in train_loader:
                        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                        optimizer.zero_grad()
                        preds = model(X_batch)
                        loss = criterion(preds, y_batch)
                        loss.backward()
                        optimizer.step()

                # --- Final validation
                model.eval()
                preds_norm, trues_norm = [], []
                with torch.no_grad():
                    for X_batch, y_batch in val_loader:
                        X_batch = X_batch.to(device)
                        out = model(X_batch).cpu().numpy()
                        preds_norm.extend(out)
                        trues_norm.extend(y_batch.numpy())

                preds = inverse_transform_predictions(np.array(preds_norm), method=norm_method, scaler=scaler)
                trues = inverse_transform_predictions(np.array(trues_norm), method=norm_method, scaler=scaler)

                # --- Metrics
                metrics = calculate_forecast_metrics(
                    trues, preds,
                    model_name=f"N-BEATS (k={n_clusters}, c={cluster_id}, seq={seq_len})"
                )
                metrics.update({
                    "Seq_len": seq_len,
                    "n_clusters": n_clusters,
                    "cluster_id": cluster_id,
                    "Best_Params": best_params
                })

                all_metrics.append(metrics)

    return pd.DataFrame(all_metrics)

# --- Run
dummy_var_nbeats = run_nbeats_with_clustering(
    feature_eng_data,
    cluster_range=(1,4),
    seq_lens=[7, 14,30,90],
    norm_method="log",  
    n_trials=20
)

print(dummy_var_nbeats)

save_df_to_downloads("dummy_var_nbeats")

Feature Set Creation for Feature Set 2 (Lag, Rolling Features, Normalization, Reverse Transformation)

In [ ]:
# Lag & Rolling logic
# -----------------
def get_lag_and_rolls(window):
    base_lags = [1, 2, 3, 7, 14, 28]
    base_rolls = [7, 14, 28]
    lags = [lag for lag in base_lags if lag <= window]
    rolls = [r for r in base_rolls if r <= window]
    return lags, rolls

# -----------------
# Feature Engineering
# -----------------
def create_lag_roll_features(df, seq_len):
    lags, rolls = get_lag_and_rolls(seq_len)
    groups = []
    for atm_id, group in df.groupby("CASHP_ID"):
        group = group.sort_values("DATE").copy()
        # Lags
        for lag in lags:
            group[f"lag_{lag}"] = group["WITHDRWLS"].shift(lag)
        # Rolling
        for r in rolls:
            group[f"roll_mean_{r}"] = group["WITHDRWLS"].shift(1).rolling(r).mean()
            group[f"roll_std_{r}"] = group["WITHDRWLS"].shift(1).rolling(r).std()
        groups.append(group)
    df_feat = pd.concat(groups).reset_index(drop=True)
    df_feat = df_feat.dropna().reset_index(drop=True)
    return df_feat


# -----------------
# Normalization helpers
# -----------------
def normalize_data(train_df, val_df, feature_cols, target_col="WITHDRWLS", method="log"):
    """
    Normalize both features (X) and target (y) using the same method.
    - log: apply log1p
    - minmax: apply MinMaxScaler
    Returns: X_train, X_val, y_train, y_val, feature_scaler, target_scaler
    """
    if method == "log":
        # Features
        X_train = np.log1p(train_df[feature_cols].values)
        X_val   = np.log1p(val_df[feature_cols].values)
        feat_scaler = None
        # Target
        y_train = np.log1p(train_df[target_col].values)
        y_val   = np.log1p(val_df[target_col].values)
        target_scaler = None

    elif method == "minmax":
        feat_scaler = MinMaxScaler()
        target_scaler = MinMaxScaler()
        # Features
        X_train = feat_scaler.fit_transform(train_df[feature_cols])
        X_val   = feat_scaler.transform(val_df[feature_cols])
        # Target
        y_train = target_scaler.fit_transform(train_df[[target_col]]).ravel()
        y_val   = target_scaler.transform(val_df[[target_col]]).ravel()

    else:
        raise ValueError("method must be 'log' or 'minmax'")

    return X_train, X_val, y_train, y_val, feat_scaler, target_scaler

def inverse_transform_predictions(y_pred, method="log", scaler=None):
    """Reverse normalization for predictions back to raw scale."""
    if method == "log":
        return np.expm1(y_pred)
    elif method == "minmax" and scaler is not None:
        return scaler.inverse_transform(y_pred.reshape(-1, 1)).ravel()
    else:
        return y_pred


Linear Regression with Feature Set 2 (Lag and Rolling Features)

In [ ]:
from sklearn.linear_model import LinearRegression
import pandas as pd

def run_linear_regression_with_new_features(
    atm,
    cluster_range=(1,3),
    seq_lens=[7,14,28],
    norm_method="log",
    test_size=0.2
):
    all_metrics = []

    for seq_len in seq_lens:
        # --- Create lag/rolling features
        df_feat = create_lag_roll_features(atm, seq_len)

        # Drop rows where target == 0 or any lag/roll == 0
        lag_roll_cols = [c for c in df_feat.columns if c.startswith(("lag_", "roll_mean_", "roll_std_"))]
        df_feat = df_feat[(df_feat["WITHDRWLS"] > 0) & (df_feat[lag_roll_cols] > 0).all(axis=1)]
        df_feat = df_feat.reset_index(drop=True)
        if df_feat.empty:
            continue

        # --- Drop dummy/calendar cols (not needed here)
        drop_cols = [c for c in df_feat.columns 
                     if c.startswith(("dow_", "dom_", "month_", "day_school_holiday", "week_part"))]
        df_feat = df_feat.drop(columns=drop_cols, errors="ignore")

        # --- Clustering
        clustered_results = cluster_atms_range(df_feat, cluster_range=cluster_range)

        for n_clusters, (df_clustered, _) in clustered_results.items():
            for cluster_id in sorted(df_clustered["cluster"].dropna().unique()):
                cluster_df = df_clustered[df_clustered["cluster"] == cluster_id].copy()

                # Drop zero-contaminated rows at cluster level
                cluster_df = cluster_df[(cluster_df["WITHDRWLS"] > 0) & (cluster_df[lag_roll_cols] > 0).all(axis=1)]
                cluster_df = cluster_df.reset_index(drop=True)
                if cluster_df.empty:
                    continue

                # --- Time split
                split_idx = int(len(cluster_df) * (1 - test_size))
                train_df, val_df = cluster_df.iloc[:split_idx], cluster_df.iloc[split_idx:]

                # --- Features: drop IDs + target + cluster
                feature_cols = [c for c in cluster_df.columns 
                                if c not in ["CASHP_ID", "DATE", "WITHDRWLS", "cluster"]]

                # --- Normalize features + target
                X_train, X_val, y_train, y_val, feat_scaler, target_scaler = normalize_data(
                    train_df, val_df, feature_cols, target_col="WITHDRWLS", method=norm_method
                )

                # --- Train model
                model = LinearRegression()
                model.fit(X_train, y_train)

                # --- Predict
                y_pred_norm = model.predict(X_val)

                # --- Reverse-transform
                y_pred_raw = inverse_transform_predictions(y_pred_norm, method=norm_method, scaler=target_scaler)
                y_true_raw = inverse_transform_predictions(y_val, method=norm_method, scaler=target_scaler)

                # --- Evaluate
                metrics = calculate_forecast_metrics(
                    y_true_raw, y_pred_raw,
                    model_name=f"LR (seq={seq_len}, k={n_clusters}, c={cluster_id})"
                )
                metrics.update({
                    "Seq_Length": seq_len,
                    "Num_Clusters": n_clusters,
                    "Cluster_ID": cluster_id,
                    "Normalization": norm_method
                })
                all_metrics.append(metrics)

    return pd.DataFrame(all_metrics)

# --- Run & Save
rolling_features_lin_reg = run_linear_regression_with_new_features(
    atm,
    cluster_range=(1,4),
    seq_lens=[7,14,28,84],
    norm_method="log"
)

print(rolling_features_lin_reg)
save_df_to_downloads("rolling_features_lin_reg")

XGBoost with Feature Set 2 (Lag and Rolling Features)

In [ ]:
import xgboost as xgb
import optuna
import pandas as pd

def run_xgboost_with_new_features(
    atm,
    cluster_range=(1,3),
    seq_lens=[7,14,28],
    n_trials=50,
    norm_method="log",
    test_size=0.2
):
    all_metrics = []

    for seq_len in seq_lens:
        # Create lag/rolling features
        df_feat = create_lag_roll_features(atm, seq_len)

        # Drop rows where target == 0 or any lag/roll == 0
        lag_roll_cols = [c for c in df_feat.columns if c.startswith(("lag_", "roll_mean_", "roll_std_"))]
        df_feat = df_feat[(df_feat["WITHDRWLS"] > 0) & (df_feat[lag_roll_cols] > 0).all(axis=1)]
        df_feat = df_feat.reset_index(drop=True)
        if df_feat.empty:
            continue

        # Drop dummy/calendar columns
        drop_cols = [c for c in df_feat.columns 
                     if c.startswith(("dow_", "dom_", "month_", "day_school_holiday", "week_part"))]
        df_feat = df_feat.drop(columns=drop_cols, errors="ignore")
        
        # Clustering
        clustered_results = cluster_atms_range(df_feat, cluster_range=cluster_range)

        for n_clusters, (df_clustered, _) in clustered_results.items():
            for cluster_id in sorted(df_clustered["cluster"].dropna().unique()):
                cluster_df = df_clustered[df_clustered["cluster"] == cluster_id].copy()

                # Drop zero-contaminated rows again at cluster level
                cluster_df = cluster_df[(cluster_df["WITHDRWLS"] > 0) & (cluster_df[lag_roll_cols] > 0).all(axis=1)]
                cluster_df = cluster_df.reset_index(drop=True)
                if cluster_df.empty:
                    continue

                # Train/val split
                split_idx = int(len(cluster_df) * (1 - test_size))
                train_df, val_df = cluster_df.iloc[:split_idx], cluster_df.iloc[split_idx:]

                # Features
                feature_cols = [c for c in cluster_df.columns 
                                if c not in ["CASHP_ID", "DATE", "WITHDRWLS", "cluster"]]
                # Normalize
                X_train, X_val, y_train, y_val, feat_scaler, target_scaler = normalize_data(
                    train_df, val_df, feature_cols, target_col="WITHDRWLS", method=norm_method
                )

                trial_metrics = []

                # Optuna objective
                def objective(trial):
                    params = {
                        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
                        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.1, log=True),
                        "max_depth": trial.suggest_int("max_depth", 3, 10),
                        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
                        "gamma": trial.suggest_float("gamma", 0, 5),
                        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
                        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
                        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
                        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
                        "random_state": 42,
                        "tree_method": "hist"
                    }

                    model = xgb.XGBRegressor(**params)
                    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

                    # --- Predict
                    y_pred_norm = model.predict(X_val)

                    # --- Reverse-transform
                    y_pred_raw = inverse_transform_predictions(y_pred_norm, method=norm_method, scaler=target_scaler)
                    y_true_raw = inverse_transform_predictions(y_val, method=norm_method, scaler=target_scaler)

                    # --- Evaluate
                    metrics = calculate_forecast_metrics(
                        y_true_raw, y_pred_raw,
                        model_name=f"XGB trial {trial.number} (seq={seq_len}, k={n_clusters}, c={cluster_id})"
                    )
                    metrics["trial"] = trial.number
                    trial_metrics.append(metrics)
                    return metrics["SMAPE"]

                # --- Run Optuna
                study = optuna.create_study(direction="minimize")
                optuna.logging.set_verbosity(optuna.logging.WARNING)
                study.optimize(objective, n_trials=n_trials)

                # --- Best trial
                best_trial = study.best_trial.number
                best_metrics = [m for m in trial_metrics if m["trial"] == best_trial][0]
                best_metrics.update({
                    "Seq_Length": seq_len,
                    "Best_Params": study.best_trial.params,
                    "Num_Clusters": n_clusters,
                    "Cluster_ID": cluster_id,
                    "Normalization": norm_method
                })
                all_metrics.append(best_metrics)

    return pd.DataFrame(all_metrics)

# --- Run & Save
rolling_features_xgboost = run_xgboost_with_new_features(
    atm,
    cluster_range=(1,4),
    seq_lens=[7,14,28,84],
    n_trials=100,
    norm_method="log"
)

print(rolling_features_xgboost)
save_df_to_downloads("rolling_features_xgboost")

LSTM with Feature Set 2 (Lag and Rolling Features)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import optuna
import pandas as pd

# -----------------
# Dataset
# -----------------
class SeqFeatureDataset(Dataset):
    def __init__(self, X, y, seq_len=30):
        self.X = X.astype(np.float32)
        self.y = y.astype(np.float32)
        self.seq_len = seq_len

    def __len__(self):
        return len(self.X) - self.seq_len

    def __getitem__(self, idx):
        X_seq = self.X[idx:idx+self.seq_len]
        y_seq = self.y[idx+self.seq_len]
        return torch.tensor(X_seq, dtype=torch.float32), torch.tensor(y_seq, dtype=torch.float32)

# -----------------
# LSTM Model
# -----------------
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers,
                            batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  
        return out.view(-1)        

# -----------------
# Runner with Feature Engineering + Clustering + Optuna
# -----------------
def run_lstm_with_rolling_and_clusters(
    atm,                       
    cluster_range=(1,4), 
    seq_lens=[7,14,28], 
    n_trials=20,
    norm_method="log"
):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    all_results = []

    for seq_len in seq_lens:
        print(f"\n--- Seq_len={seq_len} ---")

        # --- Feature engineering
        df_feat = create_lag_roll_features(atm, seq_len)

        # Drop sequences with zero target or zero lag/roll features
        lag_roll_cols = [c for c in df_feat.columns if c.startswith(("lag_", "roll_mean_", "roll_std_"))]
        df_feat = df_feat[(df_feat["WITHDRWLS"] > 0) & (df_feat[lag_roll_cols] > 0).all(axis=1)]
        df_feat = df_feat.reset_index(drop=True)
        if df_feat.empty:
            continue

        # --- Clustering
        clustered_results = cluster_atms_range(df_feat, cluster_range=cluster_range)

        for n_clusters, (df_clustered, _) in clustered_results.items():
            for cluster_id in sorted(df_clustered["cluster"].dropna().unique()):
                cluster_df = df_clustered[df_clustered["cluster"] == cluster_id].copy()

                # Drop zero-contaminated sequences again at cluster level
                cluster_df = cluster_df[(cluster_df["WITHDRWLS"] > 0) & (cluster_df[lag_roll_cols] > 0).all(axis=1)]
                cluster_df = cluster_df.reset_index(drop=True)
                if cluster_df.empty:
                    continue

                # --- Train/val split
                split_idx = int(len(cluster_df) * 0.8)
                train_df, val_df = cluster_df.iloc[:split_idx], cluster_df.iloc[split_idx:]

                # --- Features
                feature_cols = [c for c in cluster_df.columns 
                                if c not in ["CASHP_ID", "DATE", "WITHDRWLS", "cluster"]]
                # --- Normalize
                X_train, X_val, y_train, y_val, feat_scaler, target_scaler = normalize_data(
                    train_df, val_df, feature_cols, target_col="WITHDRWLS", method=norm_method
                )

                trial_metrics = []

                def objective(trial):
                    hidden_dim = trial.suggest_int("hidden_dim", 16, 128, step=16)
                    num_layers = trial.suggest_int("num_layers", 1, 3)
                    dropout    = trial.suggest_float("dropout", 0.0, 0.5)
                    lr         = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
                    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])

                    train_dataset = SeqFeatureDataset(X_train, y_train, seq_len)
                    val_dataset   = SeqFeatureDataset(X_val, y_val, seq_len)
                    train_loader  = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader    = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

                    model = LSTMModel(input_dim=X_train.shape[1],
                                      hidden_dim=hidden_dim,
                                      num_layers=num_layers,
                                      dropout=dropout).to(device)
                    criterion = nn.MSELoss()
                    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

                    # --- Training
                    for _ in range(20):
                        model.train()
                        for X_batch, y_batch in train_loader:
                            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                            optimizer.zero_grad()
                            preds = model(X_batch)
                            loss = criterion(preds, y_batch)
                            loss.backward()
                            optimizer.step()

                    # --- Validation
                    model.eval()
                    preds_norm, trues_norm = [], []
                    with torch.no_grad():
                        for X_batch, y_batch in val_loader:
                            X_batch = X_batch.to(device)
                            out = model(X_batch).cpu().numpy()
                            preds_norm.extend(out)
                            trues_norm.extend(y_batch.numpy())

                    # Reverse-transform
                    y_pred_raw = inverse_transform_predictions(np.array(preds_norm), method=norm_method, scaler=target_scaler)
                    y_true_raw = inverse_transform_predictions(np.array(trues_norm), method=norm_method, scaler=target_scaler)

                    metrics = calculate_forecast_metrics(
                        y_true_raw, y_pred_raw,
                        model_name=f"LSTM trial {trial.number} (seq={seq_len}, k={n_clusters}, c={cluster_id})"
                    )
                    metrics["trial"] = trial.number
                    trial_metrics.append(metrics)
                    return metrics["SMAPE"]

                # --- Optuna search
                optuna.logging.set_verbosity(optuna.logging.WARNING)
                study = optuna.create_study(direction="minimize")
                study.optimize(objective, n_trials=n_trials)

                # --- Best trial
                best_trial = study.best_trial.number
                best_metrics = [m for m in trial_metrics if m["trial"] == best_trial][0]
                best_metrics.update({
                    "Seq_Length": seq_len,
                    "Best_Params": study.best_trial.params,
                    "Num_Clusters": n_clusters,
                    "Cluster_ID": cluster_id,
                    "Normalization": norm_method
                })
                all_results.append(best_metrics)

    return pd.DataFrame(all_results)

# --- Run
rolling_features_lstm = run_lstm_with_rolling_and_clusters(
    atm,
    cluster_range=(1,4),
    seq_lens=[7,14,28,84],
    n_trials=20,
    norm_method="log"
)

print(rolling_features_lstm)

save_df_to_downloads("rolling_features_lstm")

N-Beats with Feature Set 2 (Lag and Rolling Features)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import optuna
import numpy as np
import pandas as pd

# -----------------
# Dataset (multivariate sequences from engineered features)
# -----------------
class SeqFeatureDataset(Dataset):
    def __init__(self, X, y, seq_len=30):
        self.X = X.astype(np.float32)
        self.y = y.astype(np.float32)
        self.seq_len = seq_len

    def __len__(self):
        return max(0, len(self.X) - self.seq_len)

    def __getitem__(self, idx):
        X_seq = self.X[idx:idx+self.seq_len]
        y_seq = self.y[idx+self.seq_len]
        return torch.tensor(X_seq, dtype=torch.float32), torch.tensor(y_seq, dtype=torch.float32)

# -----------------
# N-BEATS (simple multivariate, flatten seq -> MLP)
# -----------------
class NBeatsBlock(nn.Module):
    def __init__(self, flat_dim, hidden_dim, theta_dim, num_layers=4):
        super().__init__()
        layers = []
        in_dim = flat_dim
        for _ in range(num_layers):
            layers += [nn.Linear(in_dim, hidden_dim), nn.ReLU()]
            in_dim = hidden_dim
        self.fc = nn.Sequential(*layers)
        self.theta = nn.Linear(hidden_dim, theta_dim)

    def forward(self, x):
        x = self.fc(x)
        return self.theta(x)

class NBeatsModel(nn.Module):
    def __init__(self, input_dim, seq_len, hidden_dim=128, num_layers=4):
        super().__init__()
        flat_dim = input_dim * seq_len
        self.block = NBeatsBlock(flat_dim, hidden_dim, theta_dim=1, num_layers=num_layers)

    def forward(self, x):
        b, seq_len, dim = x.shape
        x = x.reshape(b, -1)     
        out = self.block(x)
        return out.view(-1)      

# -----------------
# Runner
# -----------------
def run_nbeats_with_rolling_and_clusters(
    atm,
    cluster_range=(1,4),
    seq_lens=[7,14,28],
    n_trials=20,
    norm_method="log",
    test_size=0.2
):
    """
    Train N-BEATS with lag/rolling features across clusters and sequence lengths.
    - Drops rows where WITHDRWLS == 0 or any lag/roll feature == 0
    - Never uses 'cluster' as a feature
    - Normalizes features + target AFTER split (log or minmax)
    - Evaluates on raw scale
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    all_results = []

    # suppress Optuna logs
    optuna.logging.set_verbosity(optuna.logging.WARNING)

    for seq_len in seq_lens:
        print(f"\n--- Seq_len={seq_len} ---")

        # --- Feature engineering
        df_feat = create_lag_roll_features(atm, seq_len)

        # Drop zero-contaminated rows (target or any lag/roll == 0)
        lag_roll_cols = [c for c in df_feat.columns if c.startswith(("lag_", "roll_mean_", "roll_std_"))]
        df_feat = df_feat[(df_feat["WITHDRWLS"] > 0) & (df_feat[lag_roll_cols] > 0).all(axis=1)]
        df_feat = df_feat.reset_index(drop=True)
        if df_feat.empty:
            continue

        # (Optional) Drop calendar dummy columns if they exist in the input
        drop_cols = [c for c in df_feat.columns if c.startswith(("dow_", "dom_", "month_", "day_school_holiday", "week_part"))]
        df_feat = df_feat.drop(columns=drop_cols, errors="ignore")

        # --- Clustering
        clustered_results = cluster_atms_range(df_feat, cluster_range=cluster_range)

        for n_clusters, (df_clustered, _) in clustered_results.items():
            for cluster_id in sorted(df_clustered["cluster"].dropna().unique()):
                cluster_df = df_clustered[df_clustered["cluster"] == cluster_id].copy()

                # Repeat zero filtering at cluster level
                cluster_df = cluster_df[(cluster_df["WITHDRWLS"] > 0) & (cluster_df[lag_roll_cols] > 0).all(axis=1)]
                cluster_df = cluster_df.reset_index(drop=True)
                if cluster_df.empty:
                    continue

                # --- Temporal split
                split_idx = int(len(cluster_df) * (1 - test_size))
                train_df, val_df = cluster_df.iloc[:split_idx], cluster_df.iloc[split_idx:]
                if len(train_df) <= seq_len or len(val_df) <= seq_len:
                    continue

                # --- Features (exclude IDs, DATE, target, and cluster)
                feature_cols = [c for c in cluster_df.columns if c not in ["CASHP_ID", "DATE", "WITHDRWLS", "cluster"]]

                # --- Normalize features + target AFTER split
                X_train, X_val, y_train, y_val, feat_scaler, target_scaler = normalize_data(
                    train_df, val_df, feature_cols, target_col="WITHDRWLS", method=norm_method
                )

                trial_metrics = []

                # -----------------
                # Optuna objective
                # -----------------
                def objective(trial):
                    hidden_dim = trial.suggest_int("hidden_dim", 32, 256, step=32)
                    num_layers = trial.suggest_int("num_layers", 2, 6)
                    lr         = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
                    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])

                    train_dataset = SeqFeatureDataset(X_train, y_train, seq_len)
                    val_dataset   = SeqFeatureDataset(X_val, y_val, seq_len)
                    if len(train_dataset) == 0 or len(val_dataset) == 0:
                        return float("inf")

                    train_loader  = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader    = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

                    model = NBeatsModel(
                        input_dim=X_train.shape[1],
                        seq_len=seq_len,
                        hidden_dim=hidden_dim,
                        num_layers=num_layers
                    ).to(device)
                    criterion = nn.MSELoss()
                    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

                    # --- Training
                    for _ in range(20):
                        model.train()
                        for X_batch, y_batch in train_loader:
                            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                            optimizer.zero_grad()
                            preds = model(X_batch)
                            loss = criterion(preds, y_batch)
                            loss.backward()
                            optimizer.step()

                    # --- Validation
                    model.eval()
                    preds_norm, trues_norm = [], []
                    with torch.no_grad():
                        for X_batch, y_batch in val_loader:
                            X_batch = X_batch.to(device)
                            out = model(X_batch).cpu().numpy()
                            preds_norm.extend(out)
                            trues_norm.extend(y_batch.numpy())

                    # Reverse-transform to raw scale
                    y_pred_raw = inverse_transform_predictions(np.array(preds_norm), method=norm_method, scaler=target_scaler)
                    y_true_raw = inverse_transform_predictions(np.array(trues_norm), method=norm_method, scaler=target_scaler)

                    metrics = calculate_forecast_metrics(
                        y_true_raw, y_pred_raw,
                        model_name=f"N-BEATS trial {trial.number} (seq={seq_len}, k={n_clusters}, c={cluster_id})"
                    )
                    metrics["trial"] = trial.number
                    trial_metrics.append(metrics)
                    return metrics["SMAPE"]

                # --- Run Optuna
                study = optuna.create_study(direction="minimize")
                study.optimize(objective, n_trials=n_trials)

                # --- Best trial
                best_trial = study.best_trial.number
                best_metrics = [m for m in trial_metrics if m["trial"] == best_trial][0]
                best_metrics.update({
                    "Seq_Length": seq_len,
                    "Best_Params": study.best_trial.params,
                    "Num_Clusters": n_clusters,
                    "Cluster_ID": cluster_id,
                    "Normalization": norm_method
                })
                all_results.append(best_metrics)

    return pd.DataFrame(all_results)

# --- Run
rolling_features_nbeats = run_nbeats_with_rolling_and_clusters(
    atm,
    cluster_range=(1,4),
    seq_lens=[7,14,28,84],
    n_trials=20,
    norm_method="log",
    test_size=0.2
)

print(rolling_features_nbeats)
save_df_to_downloads("rolling_features_nbeats")

Feature Set Creation for Feature Set 3 (Lag, Rolling Features, Normalization, Reverse Transformation)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

def normalize_data_mixed(train_df, val_df, feature_cols, target_col="WITHDRWLS", method="log"):
    """
    Normalize target and lag/roll features.
    - log: apply log1p only to lag/roll and target columns
    - minmax: scale everything
    - dummy features (0/1) stay untouched for log
    """
    # Separate feature groups
    lag_roll_cols = [c for c in feature_cols if c.startswith(("lag_", "roll_mean_", "roll_std_"))]
    other_cols = [c for c in feature_cols if c not in lag_roll_cols]

    if method == "log":
        # Features: log only lag/roll, keep dummies untouched
        X_train = np.concatenate([
            np.log1p(train_df[lag_roll_cols].values),
            train_df[other_cols].values.astype(float)
        ], axis=1)
        X_val = np.concatenate([
            np.log1p(val_df[lag_roll_cols].values),
            val_df[other_cols].values.astype(float)
        ], axis=1)
        feat_scaler = None

        # Target
        y_train = np.log1p(train_df[target_col].values)
        y_val   = np.log1p(val_df[target_col].values)
        target_scaler = None

    elif method == "minmax":
        feat_scaler = MinMaxScaler()
        target_scaler = MinMaxScaler()

        # Features
        X_train = feat_scaler.fit_transform(train_df[feature_cols])
        X_val   = feat_scaler.transform(val_df[feature_cols])

        # Target
        y_train = target_scaler.fit_transform(train_df[[target_col]]).ravel()
        y_val   = target_scaler.transform(val_df[[target_col]]).ravel()

    else:
        raise ValueError("method must be 'log' or 'minmax'")

    return X_train, X_val, y_train, y_val, feat_scaler, target_scaler

Linear Regression with Feature Set 3 (Merged Features)

In [ ]:
from sklearn.linear_model import LinearRegression
import pandas as pd

def run_linear_regression_with_both_features(
    atm,
    special_days, 
    half_days,
    cluster_range=(1,4),
    seq_lens=[7,14,28],
    norm_method="log",
    test_size=0.2
):
    all_metrics = []

    for seq_len in seq_lens:
        print(f"\n--- Seq_len={seq_len} ---")

        # 1. Add calendar + holiday features
        df_feat = feature_engineering_pipeline(atm, special_days, half_days)
        
        # 2. Add lag/rolling features
        df_feat = create_lag_roll_features(df_feat, seq_len)

        # Drop rows where target == 0 or any lag/roll == 0
        lag_roll_cols = [c for c in df_feat.columns if c.startswith(("lag_", "roll_mean_", "roll_std_"))]
        df_feat = df_feat[(df_feat["WITHDRWLS"] > 0) & (df_feat[lag_roll_cols] > 0).all(axis=1)]
        df_feat = df_feat.reset_index(drop=True)
        if df_feat.empty:
            continue

        # Clustering
        clustered_results = cluster_atms_range(df_feat, cluster_range=cluster_range)

        for n_clusters, (df_clustered, _) in clustered_results.items():
            for cluster_id in sorted(df_clustered["cluster"].dropna().unique()):
                cluster_df = df_clustered[df_clustered["cluster"] == cluster_id].copy()

                # Drop again at cluster level
                cluster_df = cluster_df[(cluster_df["WITHDRWLS"] > 0) & (cluster_df[lag_roll_cols] > 0).all(axis=1)]
                cluster_df = cluster_df.reset_index(drop=True)
                if cluster_df.empty:
                    continue

                # Temporal split
                split_idx = int(len(cluster_df) * (1 - test_size))
                train_df, val_df = cluster_df.iloc[:split_idx], cluster_df.iloc[split_idx:]

                # Features () exclude IDs, target, cluster)
                feature_cols = [
                    c for c in cluster_df.columns
                    if c not in ["CASHP_ID", "DATE", "WITHDRWLS", "cluster"]
                ]

                # Normalize
                X_train, X_val, y_train, y_val, feat_scaler, target_scaler = normalize_data_mixed(
                    train_df, val_df, feature_cols, target_col="WITHDRWLS", method=norm_method
                )

                # Train
                model = LinearRegression()
                model.fit(X_train, y_train)

                # Predict
                y_pred_norm = model.predict(X_val)

                # Reverse-transform
                y_pred_raw = inverse_transform_predictions(y_pred_norm, method=norm_method, scaler=target_scaler)
                y_true_raw = inverse_transform_predictions(y_val, method=norm_method, scaler=target_scaler)

                # Evaluate
                metrics = calculate_forecast_metrics(
                    y_true_raw, y_pred_raw,
                    model_name=f"LR (both features, seq={seq_len}, k={n_clusters}, c={cluster_id})"
                )
                metrics.update({
                    "Seq_Length": seq_len,
                    "Num_Clusters": n_clusters,
                    "Cluster_ID": cluster_id,
                    "Normalization": norm_method
                })
                all_metrics.append(metrics)

    return pd.DataFrame(all_metrics)


# Run
both_features_lin_reg = run_linear_regression_with_both_features(
    atm,
    special_days,
    half_days,
    cluster_range=(1,4),
    seq_lens=[7,14,28,84],
    norm_method="log"
)

print(both_features_lin_reg)
save_df_to_downloads("both_features_lin_reg")


XGBoost with Feature Set 3 (Merged Features)

In [ ]:
import xgboost as xgb
import optuna
import pandas as pd

def run_xgboost_with_both_features(
    atm,
    official_holidays,
    half_days,
    cluster_range=(1,4),
    seq_lens=[7,14,28],
    n_trials=50,
    norm_method="log",
    test_size=0.2
):
    all_metrics = []

    # Silence Optuna logs
    optuna.logging.set_verbosity(optuna.logging.WARNING)

    for seq_len in seq_lens:
        print(f"\n--- Seq_len={seq_len} ---")

        # 1. Add calendar + holiday features
        df_feat = feature_engineering_pipeline(atm, official_holidays, half_days)

        # 2. Add lag/rolling features
        df_feat = create_lag_roll_features(df_feat, seq_len)

        # Drop zero-contaminated rows
        lag_roll_cols = [c for c in df_feat.columns if c.startswith(("lag_", "roll_mean_", "roll_std_"))]
        df_feat = df_feat[(df_feat["WITHDRWLS"] > 0) & (df_feat[lag_roll_cols] > 0).all(axis=1)]
        df_feat = df_feat.reset_index(drop=True)
        if df_feat.empty:
            continue

        # Clustering
        clustered_results = cluster_atms_range(df_feat, cluster_range=cluster_range)

        for n_clusters, (df_clustered, _) in clustered_results.items():
            for cluster_id in sorted(df_clustered["cluster"].dropna().unique()):
                cluster_df = df_clustered[df_clustered["cluster"] == cluster_id].copy()

                # Drop again at cluster level
                cluster_df = cluster_df[(cluster_df["WITHDRWLS"] > 0) & (cluster_df[lag_roll_cols] > 0).all(axis=1)]
                cluster_df = cluster_df.reset_index(drop=True)
                if cluster_df.empty:
                    continue

                # Temporal split
                split_idx = int(len(cluster_df) * (1 - test_size))
                train_df, val_df = cluster_df.iloc[:split_idx], cluster_df.iloc[split_idx:]

                # Features ( exclude IDs, target, cluster)
                feature_cols = [
                    c for c in cluster_df.columns
                    if c not in ["CASHP_ID", "DATE", "WITHDRWLS", "cluster"]
                ]

                # Normalize (fixed version: only log-transform lag/rolls + target)
                X_train, X_val, y_train, y_val, feat_scaler, target_scaler = normalize_data_mixed(
                    train_df, val_df, feature_cols, target_col="WITHDRWLS", method=norm_method
                )

                trial_metrics = []

                # Optuna objective
                def objective(trial):
                    params = {
                        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
                        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.1, log=True),
                        "max_depth": trial.suggest_int("max_depth", 3, 10),
                        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
                        "gamma": trial.suggest_float("gamma", 0, 5),
                        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
                        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
                        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
                        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
                        "random_state": 42,
                        "tree_method": "hist"
                    }

                    model = xgb.XGBRegressor(**params)
                    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

                    # Predict
                    y_pred_norm = model.predict(X_val)

                    # Reverse-transform
                    y_pred_raw = inverse_transform_predictions(y_pred_norm, method=norm_method, scaler=target_scaler)
                    y_true_raw = inverse_transform_predictions(y_val, method=norm_method, scaler=target_scaler)

                    # Evaluate
                    metrics = calculate_forecast_metrics(
                        y_true_raw, y_pred_raw,
                        model_name=f"XGB trial {trial.number} (both features, seq={seq_len}, k={n_clusters}, c={cluster_id})"
                    )
                    metrics["trial"] = trial.number
                    trial_metrics.append(metrics)
                    return metrics["SMAPE"]

                # Run Optuna
                study = optuna.create_study(direction="minimize")
                study.optimize(objective, n_trials=n_trials)

                # Best trial
                best_trial = study.best_trial.number
                best_metrics = [m for m in trial_metrics if m["trial"] == best_trial][0]
                best_metrics.update({
                    "Seq_Length": seq_len,
                    "Best_Params": study.best_trial.params,
                    "Num_Clusters": n_clusters,
                    "Cluster_ID": cluster_id,
                    "Normalization": norm_method
                })
                all_metrics.append(best_metrics)

    return pd.DataFrame(all_metrics)

# Run
both_features_xgboost = run_xgboost_with_both_features(
    atm,
    special_days,
    half_days,
    cluster_range=(1,4),
    seq_lens=[7,14,28,84],
    n_trials=100,
    norm_method="log"
)

print(both_features_xgboost)
save_df_to_downloads("both_features_xgboost")

LSTM with Feature Set 3 (Merged Features)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import optuna
import numpy as np
import pandas as pd

# -----------------
# Dataset (with feature sequences)
# -----------------
class SeqFeatureDataset(Dataset):
    def __init__(self, X, y, seq_len=30):
        self.X = X.astype(np.float32)
        self.y = y.astype(np.float32)
        self.seq_len = seq_len

    def __len__(self):
        return len(self.X) - self.seq_len

    def __getitem__(self, idx):
        X_seq = self.X[idx:idx+self.seq_len]
        y_seq = self.y[idx+self.seq_len]
        return torch.tensor(X_seq, dtype=torch.float32), torch.tensor(y_seq, dtype=torch.float32)

# -----------------
# LSTM Model
# -----------------
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_dim, hidden_dim, num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0 
        )
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :]) 
        return out.view(-1)          

# -----------------
# Runner with both features
# -----------------
def run_lstm_with_both_features(
    atm,
    special_days,
    half_days,
    cluster_range=(1,4),
    seq_lens=[7,14,28],
    n_trials=20,
    norm_method="log"
):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    all_results = []

    # Silence Optuna logs
    optuna.logging.set_verbosity(optuna.logging.WARNING)

    for seq_len in seq_lens:
        print(f"\n--- Seq_len={seq_len} ---")

        # 1. Add calendar + holiday features
        df_feat = feature_engineering_pipeline(atm, special_days, half_days)

        # 2. Add lag/rolling features
        df_feat = create_lag_roll_features(df_feat, seq_len)

        # Drop zero-contaminated rows
        lag_roll_cols = [c for c in df_feat.columns if c.startswith(("lag_", "roll_mean_", "roll_std_"))]
        df_feat = df_feat[(df_feat["WITHDRWLS"] > 0) & (df_feat[lag_roll_cols] > 0).all(axis=1)]
        df_feat = df_feat.reset_index(drop=True)
        if df_feat.empty:
            continue

        # Clustering
        clustered_results = cluster_atms_range(df_feat, cluster_range=cluster_range)

        for n_clusters, (df_clustered, _) in clustered_results.items():
            for cluster_id in sorted(df_clustered["cluster"].dropna().unique()):
                cluster_df = df_clustered[df_clustered["cluster"] == cluster_id].copy()

                # Drop again at cluster level
                cluster_df = cluster_df[(cluster_df["WITHDRWLS"] > 0) & (cluster_df[lag_roll_cols] > 0).all(axis=1)]
                cluster_df = cluster_df.reset_index(drop=True)
                if cluster_df.empty:
                    continue

                # Train/val split
                split_idx = int(len(cluster_df) * 0.8)
                train_df, val_df = cluster_df.iloc[:split_idx], cluster_df.iloc[split_idx:]

                # Features
                feature_cols = [
                    c for c in cluster_df.columns
                    if c not in ["CASHP_ID", "DATE", "WITHDRWLS", "cluster"]
                ]

                # Normalize (log only for lags/rolls + target)
                X_train, X_val, y_train, y_val, feat_scaler, target_scaler = normalize_data_mixed(
                    train_df, val_df, feature_cols, target_col="WITHDRWLS", method=norm_method
                )

                trial_metrics = []

                # Optuna objective
                def objective(trial):
                    hidden_dim = trial.suggest_int("hidden_dim", 16, 128, step=16)
                    num_layers = trial.suggest_int("num_layers", 1, 3)
                    dropout    = trial.suggest_float("dropout", 0.0, 0.5)
                    lr         = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
                    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])

                    train_dataset = SeqFeatureDataset(X_train, y_train, seq_len)
                    val_dataset   = SeqFeatureDataset(X_val, y_val, seq_len)
                    train_loader  = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader    = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

                    model = LSTMModel(
                        input_dim=X_train.shape[1],
                        hidden_dim=hidden_dim,
                        num_layers=num_layers,
                        dropout=dropout
                    ).to(device)

                    criterion = nn.MSELoss()
                    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

                    # Training
                    for _ in range(20):
                        model.train()
                        for X_batch, y_batch in train_loader:
                            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                            optimizer.zero_grad()
                            preds = model(X_batch)
                            loss = criterion(preds, y_batch)
                            loss.backward()
                            optimizer.step()

                    # Validation
                    model.eval()
                    preds_norm, trues_norm = [], []
                    with torch.no_grad():
                        for X_batch, y_batch in val_loader:
                            X_batch = X_batch.to(device)
                            out = model(X_batch).cpu().numpy()
                            preds_norm.extend(out)
                            trues_norm.extend(y_batch.numpy())

                    # Reverse-transform
                    y_pred_raw = inverse_transform_predictions(np.array(preds_norm), method=norm_method, scaler=target_scaler)
                    y_true_raw = inverse_transform_predictions(np.array(trues_norm), method=norm_method, scaler=target_scaler)

                    metrics = calculate_forecast_metrics(
                        y_true_raw, y_pred_raw,
                        model_name=f"LSTM trial {trial.number} (both features, seq={seq_len}, k={n_clusters}, c={cluster_id})"
                    )
                    metrics["trial"] = trial.number
                    trial_metrics.append(metrics)
                    return metrics["SMAPE"]

                # Run Optuna
                study = optuna.create_study(direction="minimize")
                study.optimize(objective, n_trials=n_trials)

                # Best trial
                best_trial = study.best_trial.number
                best_metrics = [m for m in trial_metrics if m["trial"] == best_trial][0]
                best_metrics.update({
                    "Seq_Length": seq_len,
                    "Best_Params": study.best_trial.params,
                    "Num_Clusters": n_clusters,
                    "Cluster_ID": cluster_id,
                    "Normalization": norm_method
                })
                all_results.append(best_metrics)

    return pd.DataFrame(all_results)

# Run
both_features_lstm = run_lstm_with_both_features(
    atm,
    special_days,
    half_days,
    cluster_range=(1,4),
    seq_lens=[7,14,28,84],
    n_trials=100,
    norm_method="log"
)

print(both_features_lstm)
save_df_to_downloads("both_features_lstm")

N-Beats with Feature Set 3 (Merged Features)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import optuna
import numpy as np
import pandas as pd

# -----------------
# Dataset
# -----------------
class SeqFeatureDataset(Dataset):
    def __init__(self, X, y, seq_len=30):
        self.X = X.astype(np.float32)
        self.y = y.astype(np.float32)
        self.seq_len = seq_len

    def __len__(self):
        return len(self.X) - self.seq_len

    def __getitem__(self, idx):
        X_seq = self.X[idx:idx+self.seq_len]
        y_seq = self.y[idx+self.seq_len]
        return torch.tensor(X_seq, dtype=torch.float32), torch.tensor(y_seq, dtype=torch.float32)

# -----------------
# N-BEATS Model
# -----------------
class NBeatsBlock(nn.Module):
    def __init__(self, flat_dim, hidden_dim, theta_dim, num_layers=4):
        super().__init__()
        layers = []
        in_dim = flat_dim
        for _ in range(num_layers):
            layers.append(nn.Linear(in_dim, hidden_dim))
            layers.append(nn.ReLU())
            in_dim = hidden_dim
        self.fc = nn.Sequential(*layers)
        self.theta = nn.Linear(hidden_dim, theta_dim)

    def forward(self, x):
        return self.theta(self.fc(x))

class NBeatsModel(nn.Module):
    def __init__(self, input_dim, seq_len, hidden_dim=128, num_layers=4):
        super().__init__()
        flat_dim = input_dim * seq_len
        self.block = NBeatsBlock(flat_dim, hidden_dim, theta_dim=1, num_layers=num_layers)

    def forward(self, x):
        b, seq_len, dim = x.shape
        x = x.reshape(b, -1)     
        out = self.block(x)
        return out.view(-1)  

# -----------------
# Runner
# -----------------
def run_nbeats_with_both_features(
    atm,
    special_days,
    half_days,
    cluster_range=(1,4),
    seq_lens=[7,14,28],
    n_trials=20,
    norm_method="log"
):
    """
    Train N-BEATS with calendar + lag/roll features for multiple clusters and sequence lengths.
    Drops zero-contaminated sequences (target or lag/roll features).
    Suppresses Optuna trial logs.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    all_results = []

    # silence Optuna logs
    optuna.logging.set_verbosity(optuna.logging.WARNING)

    for seq_len in seq_lens:
        print(f"\n--- Seq_len={seq_len} ---")

        # 1. Add calendar + holiday features
        df_feat = feature_engineering_pipeline(atm, special_days, half_days)

        # 2. Add lag/rolling features
        df_feat = create_lag_roll_features(df_feat, seq_len)

        # Drop zero-contaminated rows
        lag_roll_cols = [c for c in df_feat.columns if c.startswith(("lag_", "roll_mean_", "roll_std_"))]
        df_feat = df_feat[(df_feat["WITHDRWLS"] > 0) & (df_feat[lag_roll_cols] > 0).all(axis=1)]
        df_feat = df_feat.reset_index(drop=True)
        if df_feat.empty:
            continue

        # Clustering
        clustered_results = cluster_atms_range(df_feat, cluster_range=cluster_range)

        for n_clusters, (df_clustered, _) in clustered_results.items():
            for cluster_id in sorted(df_clustered["cluster"].dropna().unique()):
                cluster_df = df_clustered[df_clustered["cluster"] == cluster_id].copy()

                # Drop again at cluster level
                cluster_df = cluster_df[(cluster_df["WITHDRWLS"] > 0) & (cluster_df[lag_roll_cols] > 0).all(axis=1)]
                cluster_df = cluster_df.reset_index(drop=True)
                if cluster_df.empty:
                    continue

                # Train/val split
                split_idx = int(len(cluster_df) * 0.8)
                train_df, val_df = cluster_df.iloc[:split_idx], cluster_df.iloc[split_idx:]

                # Features
                feature_cols = [
                    c for c in cluster_df.columns
                    if c not in ["CASHP_ID", "DATE", "WITHDRWLS", "cluster"]
                ]

                # Normalize
                X_train, X_val, y_train, y_val, feat_scaler, target_scaler = normalize_data_mixed(
                    train_df, val_df, feature_cols, target_col="WITHDRWLS", method=norm_method
                )

                trial_metrics = []

                def objective(trial):
                    hidden_dim = trial.suggest_int("hidden_dim", 32, 256, step=32)
                    num_layers = trial.suggest_int("num_layers", 2, 6)
                    lr         = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
                    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])

                    train_dataset = SeqFeatureDataset(X_train, y_train, seq_len)
                    val_dataset   = SeqFeatureDataset(X_val, y_val, seq_len)
                    train_loader  = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader    = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

                    model = NBeatsModel(input_dim=X_train.shape[1],
                                        seq_len=seq_len,
                                        hidden_dim=hidden_dim,
                                        num_layers=num_layers).to(device)
                    criterion = nn.MSELoss()
                    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

                    # Training
                    for _ in range(20):
                        model.train()
                        for X_batch, y_batch in train_loader:
                            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                            optimizer.zero_grad()
                            preds = model(X_batch)
                            loss = criterion(preds, y_batch)
                            loss.backward()
                            optimizer.step()

                    # Validation
                    model.eval()
                    preds_norm, trues_norm = [], []
                    with torch.no_grad():
                        for X_batch, y_batch in val_loader:
                            X_batch = X_batch.to(device)
                            out = model(X_batch).cpu().numpy()
                            preds_norm.extend(out)
                            trues_norm.extend(y_batch.numpy())

                    # Reverse-transform
                    y_pred_raw = inverse_transform_predictions(np.array(preds_norm), method=norm_method, scaler=target_scaler)
                    y_true_raw = inverse_transform_predictions(np.array(trues_norm), method=norm_method, scaler=target_scaler)

                    metrics = calculate_forecast_metrics(
                        y_true_raw, y_pred_raw,
                        model_name=f"N-BEATS trial {trial.number} (both features, seq={seq_len}, k={n_clusters}, c={cluster_id})"
                    )
                    metrics["trial"] = trial.number
                    trial_metrics.append(metrics)
                    return metrics["SMAPE"]

                # Optuna tuning
                study = optuna.create_study(direction="minimize")
                study.optimize(objective, n_trials=n_trials)

                # Best trial
                best_trial = study.best_trial.number
                best_metrics = [m for m in trial_metrics if m["trial"] == best_trial][0]
                best_metrics.update({
                    "Seq_Length": seq_len,
                    "Best_Params": study.best_trial.params,
                    "Num_Clusters": n_clusters,
                    "Cluster_ID": cluster_id,
                    "Normalization": norm_method
                })
                all_results.append(best_metrics)

    return pd.DataFrame(all_results)

# Run
both_features_nbeats = run_nbeats_with_both_features(
    atm,
    special_days,
    half_days,
    cluster_range=(1,4),
    seq_lens=[7,14,28,84],
    n_trials=20,
    norm_method="log"
)

print(both_features_nbeats)
save_df_to_downloads("both_features_nbeats")